In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Practice').getOrCreate()

In [3]:
training=spark.read.csv('test4.csv', header=True, inferSchema=True)

In [4]:
training.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|   vivek| 25|         3| 50000|
|   Aadit| 26|         4| 70000|
|Abhishek| 27|         6| 60000|
|   Harsh| 24|         2| 25000|
|   Vikas| 25|         3| 45000|
|   Fahad| 26|         4| 65000|
+--------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler= VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [11]:
output=featureassembler.transform(training)

In [12]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|   vivek| 25|         3| 50000|          [25.0,3.0]|
|   Aadit| 26|         4| 70000|          [26.0,4.0]|
|Abhishek| 27|         6| 60000|          [27.0,6.0]|
|   Harsh| 24|         2| 25000|          [24.0,2.0]|
|   Vikas| 25|         3| 45000|          [25.0,3.0]|
|   Fahad| 26|         4| 65000|          [26.0,4.0]|
+--------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [14]:
finalized_data=output.select("Independent Features","Salary")

In [15]:
finalized_data

DataFrame[Independent Features: vector, Salary: int]

In [16]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [25.0,3.0]| 50000|
|          [26.0,4.0]| 70000|
|          [27.0,6.0]| 60000|
|          [24.0,2.0]| 25000|
|          [25.0,3.0]| 45000|
|          [26.0,4.0]| 65000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression
train_data, test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [19]:
## coefficients
regressor.coefficients

DenseVector([50000.0, -28750.0])

In [20]:
### Intercepts
regressor.intercept

-1117499.9999996952

In [21]:
## prediction
pred_result= regressor.evaluate(test_data)

In [22]:
pred_result.predictions.show()

+--------------------+------+----------+
|Independent Features|Salary|prediction|
+--------------------+------+----------+
|          [25.0,3.0]| 45000|   46250.0|
|          [25.0,3.0]| 50000|   46250.0|
+--------------------+------+----------+

